In [14]:
import numpy as np
from numpy import *
import qutip
from qutip import *
from numpy.random import rand, randint
from qutip.measurement import measure, measurement_statistics
# from qutip.states import ket2dm, basis

In [15]:
class Node():
    def get_alpha(self):
        return self.alpha
    def get_children(self):
        return self.children

class DecisionTree():

    # this is an M-ary tree of depth N, i.e. it has M**(N+1) - 1 nodes in total
    # each node corresponds to a measurement history,
    # i.e. a list x_j = [k_0, k_1, ... k_j] of measurement outcomes.

    
    # initialize as as random. each node has value between 0 and 1/2 + i
    def __init__(self, M, N):
        self.alpha = 0.5*rand() + 1j*rand()
        if N == 1:
            self.children = None
        else:
            self.children = [DecisionTree(M, N-1)]*M
        #self.tree = self.makeRandTree(M, N)

    def makeZeroTree(self, M, N):
        if N == 1:
            return [0]*M
        else:
            return [self.makeZeroTree(M, N-1)] * M


    # first element is value at this location, the second is a list (the children)
    def makeRandTree(self, M, N):
        if N == 1:
            return [0.2*rand()]*M
        else:
            return [0.2*rand(), self.makeRandTree(M, N-1)] * M

    def makeCountedTree(self, M, N, count = 0):
        if N == 1:
            return list(range(count, count + M))
            count += M
        else:
            return [self.makeCountedTree(M, N-1, count)] * M

    def get_alpha(self, x):
        r = self
        for j in x:
            print(f"r: {r}")
            r = r.children[j]
            
        return r.alpha

In [16]:
# given an input state, a decision tree, and a measurement history, outputs the probability of that
# measurement history
def prob(tree, x, state):
    return 0

def loss(tree, states):
    return 0

In [20]:
np.sqrt(16)



M = 2# dimension of Hilbert space. This is a truncated Fock state, with the basis (|0>, |1>, ... |M-1>).
# Hilbert space has dimension 2**M
N = 3 # depth of circuit, including the root node
beta = 5 # this is a small parameter to ensure the coherent states are "weak"
gamma = 1e-3 # intensity of the injected states
# six states to distinguish between. these are also the ones injected, i think
code = {
    0: qutip.states.coherent(M, beta*np.exp(1j * np.atan(0.5))),
    1: qutip.states.coherent(M, beta*np.exp(-1j * np.atan(0.5)))
}

projs = [qutip.states.ket2dm(qutip.states.basis(M, j)) for j in range(M)] # single-mode projectors
projs0 = [tensor(p, identity(M)) for p in projs]
# only measures mode 0.
# this gives list of M projection operators, |j><j| \otimes I
anc = code[1]
state = tensor(code[0], anc)
[outcome, projected] = measure(state, projs0)
print(code)

tree = [1, [2, 3]]

tree2 = DecisionTree(M, N)
print("tree:")
print(f"alpha: {tree2.get_alpha([1, 0])}")

anc = coherent(M, gamma*exp(2*pi*1j*rand()))
# takes an input state, and a decision tree. yields the output state
# always add the ancillary to the tail, and measure from the head (cf. picture in article)
def process(state, tree):
    for j in range(N):
        # inject states from the several LO's: these, I assume, have known intensity but random phase
        anc = coherent(M, gamma*exp(2*pi*1j*rand()))
        state = tensor(state, anc)
        #alpha = tree(x)
        
        #k = meas


{0: Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.28366219+0.j        ]
 [-0.85768795-0.42884397j]], 1: Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.28366219+0.j        ]
 [-0.85768795+0.42884397j]]}
tree:
r: <__main__.DecisionTree object at 0x7b9a9e06b460>
r: <__main__.DecisionTree object at 0x7b9a9e06b350>
alpha: (0.19331507859476077+0.37032863197299903j)


In [85]:
print(range(0, 3))

range(0, 3)


In [57]:
print(list(range(0, 3)))

[0, 1, 2]
